# Información de BCRP

###### A continuación, vamos a extraer información del BCRP usando su api. Se realizará un proceso de automatización en el cual solo se tienen que elegir las variables que se desean obtener y seleccionar la fecha de interés a analizar.

In [1]:
# Códigos para actualizar el DashBoard Financiero

# PD04650MD : Reservas internacionales netas
# PD04692MD : Tasa de interés interbancaria S/.
# PD31893DD : Rendimiento del Bono del gobierno peruano a 10 años (en S/)
# PD31894DD : Rendimiento del Bono del gobierno peruano a 10 años (en US$)
# PD04637PD : TC Interbancario (S/ por US$) - Compra
# PD04638PD : TC Interbancario (S/ por US$) - Venta
# PD38026MD : Índice General Bursátil BVL (índice)

In [1]:
# Importamos las librerías necesarias para el trabajo
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data
import requests

In [2]:
# Elegimos la lista que deseamos trabajar
# Dashboard financiero:
lista=["PD04650MD","PD04692MD","PD31893DD","PD31894DD","PD04637PD","PD04638PD","PD38026MD"] 

In [3]:
# Este es un loop que me permite extraer todas las series de interés contenidas en "lista".
# Creamos primero un contenedor para colocar las series extraídas
d = {}

for serie in lista: 
    # A continuación, colocamos los datos requeridos para el uso del api
    formato = "json"
    fecha_inicio = "2020-10-01"
    fecha_fin = "2022-11-03"
    url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"+serie+"/"+formato+"/"+fecha_inicio+"/"+fecha_fin+"/"
    # Empezamos extrayendo la información en formato json   
    periodos = requests.get(url).json().get("periods")
    # VALORES
    dato = []
    for i in periodos:
        valores_list = i['values']
        for w in valores_list:
            w = float(w)
            dato.append(w)
            d[serie] = pd.DataFrame(dato)
    # FECHAS
    fechas = []
    for i in periodos:
        nombres = i['name']
        fechas.append(nombres)
        df_fechas = pd.DataFrame(fechas)

In [4]:
# EXPORTAR DATOS: Con este código juntaremos todas las variables extraídas.
# Dashboard financiero
data = pd.concat((df_fechas, d['PD04650MD'], d['PD04692MD'], d['PD31893DD'], d['PD31894DD'], d['PD04637PD'], d['PD04638PD'], d['PD38026MD']), axis=1, ignore_index=True)
data.rename(columns = {0:'fecha', 1:'reservas_inter',2:'interbancaria',3:'bono_soles',4:'bono_dolares',5:'tc_compra',6:'tc_venta',7:'bvl'}, inplace = True)
# Exportamos a excel
data.to_excel('DB Financiero.xlsx', sheet_name='Monedas', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None)

In [5]:
data

,fecha,reservas_inter,interbancaria,bono_soles,bono_dolares,tc_compra,tc_venta,bvl
0,01.Oct.20,73644.158334,0.25,4.30,1.864,3.601833,3.604667,17842.81
1,02.Oct.20,73497.376912,0.25,4.32,1.862,3.618000,3.620000,17911.56
2,05.Oct.20,73493.643778,0.25,4.27,1.866,3.603167,3.607667,17953.22
3,06.Oct.20,73401.240830,0.25,4.25,1.847,3.581833,3.584167,18128.92
4,07.Oct.20,73509.732613,0.25,4.22,1.815,3.575500,3.578333,17958.64
...,...,...,...,...,...,...,...,...
522,28.Oct.22,74877.947904,7.00,8.41,5.990,3.977000,3.980167,20967.74
523,31.Oct.22,73827.722757,7.00,8.23,5.926,3.971833,3.975333,21095.21
524,02.Nov.22,74319.834957,7.00,8.24,6.017,3.988333,3.990500,20866.42
525,03.Nov.22,74345.545109,7.00,NaN,NaN,3.960000,3.967750,21003.28
